## Note to run ollama first

In [1]:
import dspy

lm = dspy.LM('ollama_chat/llama3:8b', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Quick test

In [2]:
lm("Say this is a test!", temperature=0.7)  # => ['This is a test!']
lm(messages=[{"role": "user", "content": "Say this is a test!"}])  # => ['This is a test!']


["It looks like someone's trying to see if I'm paying attention! Well, I'm happy to report that I am indeed paying attention and ready to respond. So, what's the next question or statement?"]

## Module
- Math

In [3]:
math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")


Prediction(
    reasoning='The only way to get a sum of 2 is if both dice show 1, which has a probability of (1/6) × (1/6) = 1/36.',
    answer=0.02777777777777778
)

- RAG

In [4]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

rag = dspy.ChainOfThought('context, question -> response')

question = "What's the name of the castle that David Gregory inherited?"
rag(context=search_wikipedia(question), question=question)


Prediction(
    reasoning='The question is asking about the name of the castle that David Gregory inherited. According to the context provided, it can be found in [1]. The relevant sentence states: "He inherited Kinnairdy Castle in 1664."',
    response='Kinnairdy Castle'
)

- Classification

In [5]:
from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal['positive', 'negative', 'neutral'] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
classify(sentence="This book was super fun to read, though not the last chapter.")


Prediction(
    sentiment='positive',
    confidence=0.8
)

- Information Extraction

In [6]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)


Apple Inc. Announces iPhone 14
['Apple Inc. announced its latest iPhone 14 today.', 'The CEO, Tim Cook, highlighted its new features in a press release.']
[{'entity': 'Apple Inc.', 'type': 'ORGANIZATION'}, {'entity': 'iPhone 14', 'type': 'PRODUCT'}, {'entity': 'Tim Cook', 'type': 'PERSON'}]


- Multi-stage Pipelines

In [8]:
class Outline(dspy.Signature):
    """Outline a thorough overview of a topic."""

    topic: str = dspy.InputField()
    title: str = dspy.OutputField()
    sections: list[str] = dspy.OutputField()
    section_subheadings: dict[str, list[str]] = dspy.OutputField(desc="mapping from section headings to subheadings")

class DraftSection(dspy.Signature):
    """Draft a top-level section of an article."""

    topic: str = dspy.InputField()
    section_heading: str = dspy.InputField()
    section_subheadings: list[str] = dspy.InputField()
    content: str = dspy.OutputField(desc="markdown-formatted section")

class DraftArticle(dspy.Module):
    def __init__(self):
        self.build_outline = dspy.ChainOfThought(Outline)
        self.draft_section = dspy.ChainOfThought(DraftSection)

    def forward(self, topic):
        outline = self.build_outline(topic=topic)
        sections = []
        for heading, subheadings in outline.section_subheadings.items():
            section, subheadings = f"## {heading}", [f"### {subheading}" for subheading in subheadings]
            section = self.draft_section(topic=outline.title, section_heading=section, section_subheadings=subheadings)
            sections.append(section.content)
        return dspy.Prediction(title=outline.title, sections=sections)

draft_article = DraftArticle()
article = draft_article(topic="World Cup 2002")


In [9]:
print(article)


Prediction(
    title='World Cup 2002: A Review',
    sections=['### Background and History\nThe 2002 FIFA World Cup was the 17th edition of the tournament, and it marked a significant milestone in the history of football. The decision to host the tournament in two countries was a unique one, reflecting the growing importance of Asian football on the global stage.\n\n### Host Countries\nSouth Korea and Japan were chosen as co-hosts for the tournament, with the aim of promoting football development and unity across the region. The event was seen as a major opportunity for both countries to showcase their capabilities and infrastructure, and it played an important role in boosting the profile of Asian football globally.', '### Format and Schedule\nThe 2002 FIFA World Cup was the 17th edition of the tournament, featuring 32 national teams from around the world. The tournament took place from May 31 to July 30, 2002, in South Korea and Japan. The format remained the same as previous editio

- Agents

In [10]:
# llama3:8b yields 5.77711111111111, which is incorrect !!!
# let's switch to gpt-4o-mini

dspy.configure(lm=dspy.LM('openai/gpt-4o-mini'))

def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)


5765.0
